<a href="https://colab.research.google.com/github/dan-a-iancu/airm/blob/master/Project_Portfolio_Management/Project_Portfolio_Management.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook implements the solution to the **Project Portfolio Management** mini-case. It assumes you are familiar with the case and the model.

____
# Basic Setup

Import useful modules, read the data and store it in data frames, and set up some useful Python lists. You may want to expand this section and make sure you understand how the data is organized, and also read the last part where the Python lists are created, as these may be very useful when you build your model.

In [1]:
#@markdown We first import some useful modules. 

# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# import numpy
import numpy as np
import urllib.request  # for file downloading

# Import pandas for data-frames
import pandas as pd
pd.options.display.max_rows = 15
pd.options.display.float_format = "{:,.2f}".format

from IPython.display import display

# Make sure Matplotlib runs inline, for nice figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
import matplotlib.ticker as ticker 

# install Gurobi (our linear optimization solver)
!pip install -i https://pypi.gurobi.com gurobipy
from gurobipy import *

# some modules to create local directories for CBC (to avoid issues with solving multiple models)
import os
def new_local_directory(name):
    full_path = os.path.join(".", name)
    os.makedirs(full_path, exist_ok=True)
    return full_path

# install the latest version of seaborn for nicer graphics
#!pip install --prefix {sys.prefix} seaborn==0.11.0  &> /dev/null
import seaborn as sns

# Ignore useless some warnings
import warnings
warnings.simplefilter(action="ignore")

print("Completed successfully!")

Looking in indexes: https://pypi.gurobi.com
Completed successfully!


## Load the case data into Pandas data frames

We first download an Excel file with all the data from Github.

In [2]:
#@markdown Download the entire data as an Excel file from Github

url_Excel = 'https://github.com/dan-a-iancu/airm/blob/master/Project_Portfolio_Management/Project_Data.xlsx?raw=true'
local_file = "Portfolio_Project_Data.xlsx"   # name of local file where you want to store the downloaded file
urllib.request.urlretrieve(url_Excel, local_file)    # download from website and save it locally

('Portfolio_Project_Data.xlsx', <http.client.HTTPMessage at 0x7ff242d287d0>)

Read in and store the data in suitable dataframes.

In [3]:
#@markdown Create a dataframe with the information on projects

# Read in the information about the available projects
projectData = pd.read_excel("Portfolio_Project_Data.xlsx", sheet_name = "Projects", index_col = 0)

# Have a look:
display(projectData)


,Capital_Required,Location,Impact,Risks,Engineers,Field_Workers,Support
Project,,,,,,,
A,4015,South America,10,10,15,80,15
B,3623,South America,7,10,5,35,15
C,3847,South America,4,7,10,30,10
D,4058,Asia,4,4,5,35,4
E,11182,Asia,10,7,10,50,5
F,4858,Asia,4,4,5,45,5
G,5379,Asia,4,4,5,40,5
H,16203,Africa,10,7,5,100,15
I,1839,Africa,1,4,2,70,10


In [4]:
#@markdown Create a dataframe with the information on available resources (this is useful in **Q5**)

# Read in the information about the available projects
resourceData = pd.read_excel("Portfolio_Project_Data.xlsx", sheet_name = "Resources", index_col = 0)

# Have a look:
display(resourceData)

,January-March,April-June,July-September,October-December
Engineers,20,20,12,20
Field_Workers,120,150,150,120
Support,50,40,40,50


Also set up any other problem data/parameters, such as the initial capital available.

In [5]:
initialCapital = 25000

## Create Python lists based on the data-frames

__NOTE__: Make sure you understand what the __lists__ created here are! These will be very helpful when creating the model.

In [6]:
#@markdown Some useful lists for building all the models

# the list with project names (A,B, ...)
allProjects = list(projectData.index)
print("This is the list of all the project names:")
print(allProjects)

# the unique locations / continents
allLocations = list(projectData["Location"].unique())
print("\nThis is the list of unique locations:")
print(allLocations)

This is the list of all the project names:
['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']

This is the list of unique locations:
['South America', 'Asia', 'Africa']


In [7]:
#@markdown The following lists will be useful in **Q5**

# the list with periods when the projects could be scheduled
allPeriods = list(resourceData.columns)
print("These are periods when the projects could be scheduled:")
print(allPeriods)

# the types of resources needed
allResources = list(resourceData.index)
print("\nThese are the unique resources needed to execute the projects:")
print(allResources)

These are periods when the projects could be scheduled:
['January-March', 'April-June', 'July-September', 'October-December']

These are the unique resources needed to execute the projects:
['Engineers', 'Field_Workers', 'Support']


_____
# **Q1**

**Create an empty model**

In [8]:
#@title Create the model
ProjectSelectionModel = Model("Funding Projects")

Restricted license - for non-production use only - expires 2023-10-25


**Define the Decision Variables**<br>
We have a **binary** decision for whether to select each project or not. We can add one such variable for each project using Gurobi's ``addVars`` method. Note that  specify the **type** of the decision using `vtype`.

In [9]:
#@title Add the decision variables
fund_project = ProjectSelectionModel.addVars(allProjects, vtype = GRB.BINARY, name="Fund")

**Calculate and add the objective function**
The objective corresponds to the total impact achieved on all the continents.

In [10]:
#@title Add the objective
ProjectSelectionModel.setObjective( 
    quicksum(fund_project[p]*projectData["Impact"][p] for p in allProjects), GRB.MAXIMIZE )

**Add the Constraints**

The only constraint here is that the capital used should not exceed the initial capital available.

In [11]:
#@title Add the constraints
# we only have one constraint, that "capital used <= capital available"
ProjectSelectionModel.addConstr( 
    quicksum(fund_project[p]*projectData["Capital_Required"][p] for p in allProjects) 
    <= initialCapital, name = "Initial_capital_avail" )

<gurobi.Constr *Awaiting Model Update*>

**Solve the model**

In [12]:
#@markdown Select whether to run the [Gurobi](https://www.gurobi.com/) optimization algorithms silently (no output details)
run_silently = False #@param {type:"boolean"}

if run_silently:
    ProjectSelectionModel.setParam('OutputFlag',0)
else:
    ProjectSelectionModel.setParam('OutputFlag',1)

ProjectSelectionModel.optimize()
print('\nSolved the optimization problem...')

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 1 rows, 10 columns and 10 nonzeros
Model fingerprint: 0x33bf829d
Variable types: 0 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [2e+03, 2e+04]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+04, 2e+04]
Found heuristic solution: objective 30.0000000
Presolve removed 1 rows and 10 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.03 seconds (0.00 work units)
Thread count was 1 (of 2 available processors)

Solution count 2: 32 30 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.200000000000e+01, best bound 3.200000000000e+01, gap 0.0000%

Solved the optimization problem...


**Print the optimal objective and optimal solution**

In [13]:
#@title Print the solution
print("Projects Funded: ")
for p in allProjects:
    if fund_project[p].X == 1:
        print("Project ",p)

print("Optimal total impact: {:.2f}".format(ProjectSelectionModel.objVal))

Projects Funded: 
Project  A
Project  B
Project  C
Project  E
Project  I
Optimal total impact: 32.00


**Calculate and print some additional information about the solution**

We calculate the "bang-for-the-buck" for each project, and display it together with the solution as a dataframe. We also calculate the impact achieved on each continent.

In [14]:
#@markdown Calculate and print additional information

# change the precision on pandas dataframes
pd.options.display.float_format = "{:,.5f}".format

# create a dataframe with the optimal decisions for each project
df_results = pd.DataFrame({"Fund_Project?" : \
                           [np.int(fund_project[p].X) for p in allProjects], \
                            "Bang-for-buck" : \
                           [ projectData["Impact"][p]/projectData["Capital_Required"][p] for p in allProjects]}, \
                          index = allProjects)
display(df_results)

# impact by continent
print('\n\n{:<50}\nImpact achieved on each continent:'.format("="*80))
for loc in allLocations:
    print("   {:<15} : {:,.2f}".format(loc, sum(projectData["Impact"][p]*fund_project[p].X \
                                                for p in allProjects if projectData["Location"][p]==loc)) )

,Fund_Project?,Bang-for-buck
A,1,0.00249
B,1,0.00193
C,1,0.00104
D,0,0.00099
E,1,0.00089
F,0,0.00082
G,0,0.00074
H,0,0.00062
I,1,0.00054
J,0,0.00048




Impact achieved on each continent:
   South America   : 21.00
   Asia            : 10.00
   Africa          : 1.00


Store the optimal impact obtained in **Q1**.

In [15]:
impact_Q1 = ProjectSelectionModel.objVal  # optimal value Q1

## Create a few useful functions

To help with subsequent parts of the problem, we also add all the steps above inside a **function** that creates and returns a generic model like the one we created in **Q1**, together with all the decision variables and constraints.

In [16]:
#@title A function that generates a model like the one in **Q1**
def create_model_like_in_Q1():
    # create the model
    ProjectSelectionModel = Model("Funding Projects")

    #@markdown Decision variables
    # one binary decision for each project
    fund_project = ProjectSelectionModel.addVars(allProjects, vtype = GRB.BINARY, name="Fund")

    #@markdown Objective
    # calculate the net impact objective and add it to the model
    ProjectSelectionModel.setObjective( 
      quicksum(fund_project[p]*projectData["Impact"][p] for p in allProjects), GRB.MAXIMIZE )

    #@markdown Constraints
    # add constraint that "capital used <= capital available"
    constraints = ProjectSelectionModel.addConstr( 
        quicksum(fund_project[p]*projectData["Capital_Required"][p] for p in allProjects) 
        <= initialCapital, name = "Initial_capital_avail" )

    # return the model, the decision variables and the constraints
    return ProjectSelectionModel,  fund_project, constraints

In [17]:
#@title A function that prints a solution and some useful information
def print_solution(fund_project):
      # change the precision on pandas dataframes
      pd.options.display.float_format = "{:,.5f}".format

      # create a dataframe with the optimal decisions for each project
      df_results = pd.DataFrame({"Fund_Project?" : \
                                [np.int(fund_project[p].X) for p in allProjects]}, \
                                index = allProjects)
      display(df_results)

      # impact by continent
      print('\n\n{:<50}\nImpact achieved on each continent:'.format("="*80))
      for loc in allLocations:
          print("   {:<15} : {:,.2f}".format(loc, sum(projectData["Impact"][p]*fund_project[p].X \
                                                      for p in allProjects if projectData["Location"][p]==loc)) )

______
# **Q2**
Before running this section, make sure you have run all the previous sections of the Colab file.

Re-recreate an identical model to the one from **Q1** and store the model, the decision variables and the constraints.

In [18]:
#@title Create a model like the one in Q1
ProjectSelectionModel, fund_project, budget_constraint = \
       create_model_like_in_Q1()

Add a new decision variable **Z** to the model, meant to capture the minimum impact on either continent.

In [19]:
#@title Add a new decision variable **Z**
Z = ProjectSelectionModel.addVar(name="min_impact")

Set the objective to maximize the new variable **Z**.

In [20]:
#@title Set the objective to maximize **Z**
ProjectSelectionModel.setObjective(Z, GRB.MAXIMIZE)

Add constraints that **Z** should be less than or equal to the impact on each continent.

In [21]:
#@title Add constraints that **Z** cannot exceed the impact on each continent
for loc in allLocations:
    ProjectSelectionModel.addConstr( Z <= quicksum(projectData["Impact"][p]*fund_project[p] \
                                                for p in allProjects if projectData["Location"][p]==loc))

Solve the new model.

In [22]:
#@title Solve the model 
#@markdown Select whether to run the [Gurobi](https://www.gurobi.com/) optimization algorithms silently (no output details)
run_silently = True #@param {type:"boolean"}

if run_silently:
    ProjectSelectionModel.setParam('OutputFlag',0)
else:
    ProjectSelectionModel.setParam('OutputFlag',1)

ProjectSelectionModel.optimize()
print('\nSolved the optimization problem...')


Solved the optimization problem...


Print information about the solution.

In [23]:
#@markdown Print the solution
# the objective in this problem
print("The largest impact that can be simultaneously achieved on each continent: {}".\
      format(ProjectSelectionModel.objVal))

# other useful information about the solution
print_solution(fund_project)

The largest impact that can be simultaneously achieved on each continent: 5.0


,Fund_Project?
A,1
B,0
C,0
D,1
E,0
F,0
G,1
H,0
I,1
J,1




Impact achieved on each continent:
   South America   : 10.00
   Asia            : 8.00
   Africa          : 5.00


Store the optimal value of the problem --- this is useful subsequently, e.g., in **Q3**.

In [24]:
#@markdown Store the optimal value for later use (in **Q3**)
# store the optimal value
opt_value_Q2 = ProjectSelectionModel.objVal

______
# **Q3**
Before running this section, make sure you have run all the previous sections of the Colab file.

Re-recreate an identical model to the one from **Q1** and store the model, the decision variables and the constraints.

In [25]:
#@title Create a model like the one in Q1
ProjectSelectionModel, fund_project, budget_constraint = \
       create_model_like_in_Q1()

Add constraints that the impact in each location should be greater than or equal to the (optimal) minimum impact calculated in **Q2**.

In [26]:
#@title Add constraints that the impact on each continent is at least the minimum calculated in **Q2**
for loc in allLocations:
    ProjectSelectionModel.addConstr( quicksum(projectData["Impact"][p]*fund_project[p] \
                                              for p in allProjects if projectData["Location"][p]==loc) \
                                    >= opt_value_Q2)

Solve the new model.

In [27]:
#@markdown Select whether to run the [Gurobi](https://www.gurobi.com/) optimization algorithms silently (no output details)
run_silently = True #@param {type:"boolean"}

if run_silently:
    ProjectSelectionModel.setParam('OutputFlag',0)
else:
    ProjectSelectionModel.setParam('OutputFlag',1)

ProjectSelectionModel.optimize()
print('\nSolved the optimization problem...')


Solved the optimization problem...


Print information about the solution.

In [28]:
#@markdown Print the solution
# the objective in this problem
print("The largest total cumulative impact that can be achieved while ensuring that each continent has impact of at least {} is : {}".\
      format(opt_value_Q2, ProjectSelectionModel.objVal))

# other useful information about the solution
print_solution(fund_project)

The largest total cumulative impact that can be achieved while ensuring that each continent has impact of at least 5.0 is : 23.0


,Fund_Project?
A,1
B,0
C,0
D,0
E,0
F,1
G,1
H,0
I,1
J,1




Impact achieved on each continent:
   South America   : 10.00
   Asia            : 8.00
   Africa          : 5.00


______
# **Q4**
Before running this section, make sure you have run all the previous sections of the Colab file.

In [29]:
#@markdown Store the max risk score in a parameter
max_risk = 5.5

Re-recreate an identical model to the one from **Q1** and store the model, the decision variables and the constraints.

In [30]:
#@title Create a model like the one in Q1
ProjectSelectionModel, fund_project, budget_constraint = \
       create_model_like_in_Q1()

Add a constraint that the average risk score should not exceed the maximum allowed risk. Note that this constraint is of the form
>  $\frac{\sum_{p} R_p \cdot X_p}{\sum_p X_p} \leq M$

where $R_p$ is the risk score for project $p$, and $X_p \in \{0,1\}$ is the binary variable indicating whether project $p$ is selected. This is a nonlinear constraint, but it can be re-formulated in a linear way as:
>  $\sum_{p} R_p \cdot X_p \leq M \cdot \sum_p X_p$

We formulate it in this linear way below. (Formulating the first non-linear constraint above would result in an error from Gurobi.)

In [31]:
#@title Add a constraint that the average risk does not exceed max allowed
ProjectSelectionModel.addConstr( quicksum(projectData["Risks"][p]*fund_project[p] \
                                          for p in allProjects) <= \
                                max_risk*quicksum(fund_project[p] for p in allProjects))

<gurobi.Constr *Awaiting Model Update*>

Solve the new model.

In [32]:
#@markdown Select whether to run the [Gurobi](https://www.gurobi.com/) optimization algorithms silently (no output details)
run_silently = True #@param {type:"boolean"}

if run_silently:
    ProjectSelectionModel.setParam('OutputFlag',0)
else:
    ProjectSelectionModel.setParam('OutputFlag',1)

ProjectSelectionModel.optimize()
print('\nSolved the optimization problem...')


Solved the optimization problem...


Print information about the solution.

In [33]:
#@markdown Print the solution
# the objective in this problem
print("The largest total cumulative impact achieved without exceeding an average risk of {} is : {}".\
      format(max_risk, ProjectSelectionModel.objVal))

# other useful information about the solution
print_solution(fund_project)

The largest total cumulative impact achieved without exceeding an average risk of 5.5 is : 27.0


,Fund_Project?
A,1
B,0
C,1
D,1
E,0
F,1
G,1
H,0
I,1
J,0




Impact achieved on each continent:
   South America   : 14.00
   Asia            : 12.00
   Africa          : 1.00


______
# **Q6**
Before running this section, make sure you have run all the previous sections of the Colab file.

Re-recreate an identical model to the one from **Q1** and store the model, the decision variables and the constraints.

In [34]:
#@title Create a model like the one in Q1
ProjectSelectionModel, fund_project, budget_constraint = \
       create_model_like_in_Q1()

Add a new set of decision variables for whether to schedule each project in a particular period. Here, we want to have one binary decision for every project and for every potential period, so we use `addVars` to define these.

In [35]:
#@title Add binary decision variables for whether to schedule the projects in a given period
schedule_project = ProjectSelectionModel.addVars(allProjects, allPeriods, name="schedule")

Add constraints to connect the decisions on whether to fund projects with the decisions on whether to schedule projects. If we let $X_p$ denote the binary decision of whether to fund project $p$, and $Y_{p,t}$ the binary variable for whether project $p$ should be scheduled in period $t$, then the constraints we need to add here are:
> $\sum_{t=1}^T Y_{p,t} = X_p$,  for every project $p$.

In other words, if $X_p=1$ (so project $p$ is funded), it must be scheduled in one of the periods, so the sum on the left must be equal to 1. If $X_p=0$, project $p$ is not funded and it should also not be scheduled.

In [36]:
#@title Constraints that a project must be scheduled if and only if it is funded.
for p in allProjects:
    ProjectSelectionModel.addConstr( fund_project[p] == \
                                    quicksum(schedule_project[p,t] for t in allPeriods))

Add constraints on the available resources. For each resource, the usage should not exceed what is available.

In [37]:
#@title Constraints on available resources
for t in allPeriods:
    for r in allResources:
        ProjectSelectionModel.addConstr( quicksum(schedule_project[p,t]*projectData[r][p] \
                                                  for p in allProjects) <= \
                                        resourceData[t][r] )

Solve the new model.

In [38]:
#@markdown Select whether to run the [Gurobi](https://www.gurobi.com/) optimization algorithms silently (no output details)
run_silently = True #@param {type:"boolean"}

if run_silently:
    ProjectSelectionModel.setParam('OutputFlag',0)
else:
    ProjectSelectionModel.setParam('OutputFlag',1)

ProjectSelectionModel.optimize()
print('\nSolved the optimization problem...')


Solved the optimization problem...


Print information about the solution.

In [39]:
#@markdown Print the solution
# the objective in this problem
print("The largest impact that can be simultaneously achieved on each continent: {}".\
      format(ProjectSelectionModel.objVal))

# create a small dataframe to store the decisions to fund as well as the decisions to schedule
dict = {"Fund_Project" : [np.int(fund_project[p].X) for p in allProjects]}
for t in allPeriods:
   dict["Schedule "+t] = [np.int(schedule_project[p,t].X) for p in allProjects]

all_decisions_Q5 = pd.DataFrame(dict, index=allProjects)

# other useful information about the solution
display(all_decisions_Q5)

The largest impact that can be simultaneously achieved on each continent: 32.0


,Fund_Project,Schedule January-March,Schedule April-June,Schedule July-September,Schedule October-December
A,1,0,0,0,0
B,1,0,0,0,0
C,1,0,0,0,0
D,0,0,0,0,0
E,1,0,0,0,0
F,0,0,0,0,0
G,0,0,0,0,0
H,0,0,0,0,0
I,1,0,1,0,0
J,0,0,0,0,0
